# Xây dựng các tác nhân AI với bộ nhớ lâu dài bằng Cognee

Notebook này hướng dẫn cách xây dựng các tác nhân AI thông minh với khả năng bộ nhớ tiên tiến sử dụng [**cognee**](https://www.cognee.ai/) - một bộ nhớ AI mã nguồn mở kết hợp đồ thị tri thức, tìm kiếm ngữ nghĩa và quản lý phiên để tạo ra các hệ thống AI nhận thức ngữ cảnh.

## 🎯 Mục tiêu học tập

Sau khi hoàn thành hướng dẫn này, bạn sẽ hiểu cách:
- **Xây dựng đồ thị tri thức dựa trên embeddings**: Chuyển đổi văn bản không cấu trúc thành tri thức có cấu trúc, có thể truy vấn
- **Triển khai bộ nhớ phiên**: Tạo các cuộc hội thoại nhiều lượt với khả năng tự động lưu giữ ngữ cảnh
- **Lưu trữ cuộc hội thoại**: Tùy chọn lưu các tương tác quan trọng vào bộ nhớ dài hạn để tham khảo trong tương lai
- **Truy vấn bằng ngôn ngữ tự nhiên**: Truy cập và tận dụng ngữ cảnh lịch sử trong các cuộc hội thoại mới
- **Trực quan hóa bộ nhớ**: Khám phá các mối quan hệ trong đồ thị tri thức của tác nhân


## 🏗️ Những gì bạn sẽ xây dựng

Trong hướng dẫn này, chúng ta sẽ tạo một **Trợ lý Lập trình** với khả năng ghi nhớ lâu dài, bao gồm:

### 1. **Xây dựng cơ sở tri thức**
   - Tiếp nhận thông tin hồ sơ và chuyên môn của nhà phát triển
   - Xử lý các nguyên tắc và thực hành tốt nhất trong lập trình Python
   - Lưu trữ các cuộc hội thoại lịch sử giữa nhà phát triển và trợ lý AI

### 2. **Hội thoại nhận thức theo phiên**
   - Duy trì ngữ cảnh qua nhiều câu hỏi trong cùng một phiên
   - Tự động lưu trữ từng cặp câu hỏi/đáp án để truy xuất hiệu quả
   - Cung cấp các phản hồi mạch lạc, có ngữ cảnh dựa trên lịch sử hội thoại

### 3. **Bộ nhớ dài hạn**
   - Lưu trữ các cuộc hội thoại quan trọng vào bộ nhớ dài hạn
   - Truy xuất các ký ức liên quan từ cơ sở tri thức và các phiên trước để hỗ trợ tương tác mới
   - Xây dựng cơ sở tri thức ngày càng phát triển và cải thiện theo thời gian

### 4. **Truy xuất ký ức thông minh**
   - Sử dụng tìm kiếm ngữ nghĩa dựa trên đồ thị để tìm thông tin liên quan trong toàn bộ dữ liệu đã lưu trữ
   - Lọc tìm kiếm theo các nhóm dữ liệu con (thông tin nhà phát triển so với nguyên tắc)
   - Kết hợp nhiều nguồn dữ liệu để cung cấp câu trả lời toàn diện


## 📋 Yêu cầu & Cài đặt

### Yêu cầu hệ thống

Trước khi bắt đầu, hãy đảm bảo bạn có:

1. **Môi trường Python**
   - Python 3.9 hoặc cao hơn
   - Môi trường ảo (khuyến nghị)
   
2. **Redis Cache** (Yêu cầu cho Quản lý Phiên)
   - Redis cục bộ: `docker run -d -p 6379:6379 redis`
   - Hoặc sử dụng dịch vụ Redis được quản lý
   
3. **Truy cập API LLM**
   - Khóa API OpenAI hoặc các nhà cung cấp khác (xem [tài liệu](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Cấu hình cơ sở dữ liệu**
   - Không cần cấu hình theo mặc định. Cognee sử dụng cơ sở dữ liệu dựa trên tệp (LanceDB và Kuzu)
   - Tùy chọn, bạn có thể thiết lập Azure AI Search làm kho vector (xem [tài liệu](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Cấu hình môi trường

Tạo tệp `.env` trong thư mục dự án của bạn với các biến sau:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Hiểu về Kiến trúc Bộ nhớ của Cognee

### Cách Cognee Hoạt Động

Cognee cung cấp một hệ thống bộ nhớ tinh vi vượt xa việc lưu trữ đơn giản theo kiểu key-value:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Các Thành Phần Chính:

1. **Đồ thị Tri thức**: Lưu trữ các thực thể, mối quan hệ và kết nối ngữ nghĩa
2. **Vector Embeddings**: Cho phép tìm kiếm ngữ nghĩa trên toàn bộ thông tin được lưu trữ
3. **Bộ nhớ đệm Phiên**: Duy trì ngữ cảnh hội thoại trong và giữa các phiên
4. **NodeSets**: Tổ chức dữ liệu thành các danh mục logic để truy xuất mục tiêu

### Các Loại Bộ Nhớ trong Hướng Dẫn Này:

- **Bộ nhớ Dài hạn**: Lưu trữ lâu dài trong đồ thị tri thức
- **Bộ nhớ Phiên**: Ngữ cảnh hội thoại tạm thời trong bộ nhớ đệm Redis
- **Bộ nhớ Ngữ nghĩa**: Tìm kiếm tương đồng dựa trên vector trên toàn bộ dữ liệu


## 📦 Cài đặt các gói cần thiết

Cài đặt Cognee với hỗ trợ Redis để quản lý phiên:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Khởi tạo môi trường và tải thư viện

Đảm bảo rằng:
1. Redis đang chạy (ví dụ, qua Docker: `docker run -d -p 6379:6379 redis`)
2. Các biến môi trường được thiết lập trước khi nhập các module cache
3. Nếu cần, khởi động lại kernel và chạy các cell theo thứ tự

Cell sau sẽ:
1. Tải các biến môi trường từ `.env`
2. Cấu hình Cognee với các thiết lập LLM của bạn
3. Kích hoạt bộ nhớ đệm để quản lý phiên làm việc
4. Xác minh tất cả các thành phần được kết nối đúng cách


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Cấu hình Thư mục Lưu trữ

Cognee sử dụng hai thư mục riêng biệt cho các hoạt động của nó:
- **Data Root**: Lưu trữ các tài liệu đã nhập và dữ liệu đã xử lý
- **System Root**: Chứa cơ sở dữ liệu đồ thị tri thức và siêu dữ liệu hệ thống

Chúng ta sẽ tạo các thư mục riêng biệt cho hướng dẫn này như sau:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Đặt lại trạng thái bộ nhớ

Trước khi chúng ta bắt đầu xây dựng hệ thống bộ nhớ, hãy đảm bảo rằng chúng ta đang bắt đầu từ đầu.

> 💡 **Mẹo**: Bạn có thể bỏ qua bước này nếu muốn giữ lại các bộ nhớ hiện có từ các lần chạy trước khi sử dụng notebook này sau.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Phần 1: Xây dựng cơ sở tri thức

### Nguồn dữ liệu cho Trợ lý Nhà phát triển của chúng ta

Chúng ta sẽ thu thập ba loại dữ liệu để tạo ra một cơ sở tri thức toàn diện:

1. **Hồ sơ nhà phát triển**: Chuyên môn cá nhân và nền tảng kỹ thuật
2. **Các thực hành tốt nhất về Python**: Triết lý Zen của Python cùng các hướng dẫn thực tiễn
3. **Các cuộc hội thoại lịch sử**: Các phiên hỏi đáp trước đây giữa nhà phát triển và trợ lý AI

Dữ liệu đa dạng này cho phép tác nhân của chúng ta:
- Hiểu được bối cảnh kỹ thuật của người dùng
- Áp dụng các thực hành tốt nhất trong các khuyến nghị
- Học hỏi từ những tương tác thành công trước đó


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Xử lý Dữ liệu thành Đồ thị Tri thức

Bây giờ chúng ta sẽ chuyển đổi văn bản thô thành một bộ nhớ có cấu trúc. Quá trình này:

1. **Thêm dữ liệu vào NodeSets**: Tổ chức thông tin thành các danh mục hợp lý
   - `developer_data`: Hồ sơ nhà phát triển và các cuộc trò chuyện
   - `principles_data`: Các thực hành tốt nhất và hướng dẫn Python

2. **Chạy Cognify Pipeline**: Trích xuất thực thể, mối quan hệ và tạo embeddings
   - Xác định các khái niệm chính
   - Tạo kết nối ngữ nghĩa giữa các thông tin liên quan
   - Tạo vector embeddings

Quá trình này có thể mất một chút thời gian khi LLM xử lý văn bản và xây dựng cấu trúc đồ thị:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Trực quan hóa Đồ thị Kiến thức

Hãy khám phá cấu trúc của đồ thị kiến thức của chúng ta. Hình ảnh trực quan hiển thị:
- **Nút**: Các thực thể được trích xuất từ văn bản (khái niệm, công nghệ, con người)
- **Cạnh**: Các mối quan hệ và kết nối giữa các thực thể
- **Cụm**: Các khái niệm liên quan được nhóm lại theo sự tương đồng ngữ nghĩa

Mở tệp HTML đã tạo trong trình duyệt của bạn để khám phá đồ thị một cách tương tác:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Làm giàu trí nhớ với Memify

Hàm `memify()` phân tích đồ thị tri thức và tạo ra các quy tắc thông minh về dữ liệu. Quá trình này:
- Xác định các mẫu và thực tiễn tốt nhất
- Tạo ra các hướng dẫn có thể hành động dựa trên nội dung
- Thiết lập mối quan hệ giữa các lĩnh vực tri thức khác nhau

Những quy tắc này giúp tác nhân đưa ra quyết định sáng suốt hơn khi trả lời câu hỏi. Việc chụp một hình ảnh trực quan thứ hai giúp bạn so sánh cách đồ thị trở nên dày đặc hơn sau khi được làm giàu.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Phần 2: Truy xuất thông tin thông minh

### Minh họa 1: Tích hợp kiến thức từ nhiều tài liệu

Bây giờ khi đồ thị tri thức của chúng ta đã được xây dựng, hãy kiểm tra cách Cognee kết hợp thông tin từ nhiều nguồn để trả lời các câu hỏi phức tạp.

Truy vấn đầu tiên minh họa:
- **Hiểu ngữ nghĩa**: Tìm các khái niệm liên quan ngay cả khi không được đề cập rõ ràng
- **Tham chiếu chéo**: Kết hợp hồ sơ nhà phát triển với các nguyên tắc Python
- **Lý luận theo ngữ cảnh**: Áp dụng các thực hành tốt nhất vào các triển khai cụ thể

### Minh họa 2: Tìm kiếm có lọc với NodeSets

Truy vấn thứ hai cho thấy cách nhắm mục tiêu vào các tập hợp con cụ thể của đồ thị tri thức:
- Sử dụng tham số `node_name` để chỉ tìm kiếm trong `principles_data`
- Cung cấp câu trả lời tập trung từ một miền tri thức cụ thể
- Hữu ích khi bạn cần thông tin chuyên biệt theo miền


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Phần 3: Thiết lập Quản lý Phiên

### Kích hoạt Bộ nhớ Hội thoại

Quản lý phiên rất quan trọng để duy trì ngữ cảnh qua nhiều tương tác. Ở đây chúng ta sẽ:

1. **Khởi tạo Ngữ cảnh Người dùng**: Tạo hoặc truy xuất hồ sơ người dùng để theo dõi phiên
2. **Cấu hình Bộ nhớ Cache**: Kết nối với Redis để lưu trữ lịch sử hội thoại
3. **Kích hoạt Biến Phiên**: Thiết lập các biến ngữ cảnh duy trì qua các truy vấn

> ⚠️ **Quan trọng**: Điều này yêu cầu Redis đang chạy và `CACHING=true` trong môi trường của bạn


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Hàm Hỗ Trợ: Xem Lịch Sử Phiên

Hàm tiện ích này cho phép chúng ta kiểm tra lịch sử hội thoại được lưu trữ trong Redis. Nó hữu ích cho:
- Gỡ lỗi quản lý phiên
- Xác minh rằng các hội thoại đang được lưu vào bộ nhớ đệm
- Hiểu rõ ngữ cảnh nào đang có sẵn cho tác nhân


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Phiên 1: Phòng thí nghiệm hỗ trợ bất đồng bộ — Câu hỏi đầu tiên

Bắt đầu phiên `async-support-lab` bằng cách hỏi về các mẫu asyncio thân thiện với việc thu thập dữ liệu từ xa cho một trình thu thập dữ liệu web quy mô lớn. Biểu đồ đã biết về asyncio, aiohttp và các thực hành giám sát, vì vậy câu trả lời nên phản ánh các cuộc trò chuyện trước đó đồng thời điều chỉnh câu trả lời cho câu hỏi mới.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Kiểm tra Bộ nhớ Phiên 1 Sau Lần Trao Đổi Đầu Tiên

Chạy `show_history(session_1)` ngay sau câu hỏi ban đầu xác nhận rằng Cognee đã ghi cả phần gợi ý và phần hoàn thành vào Redis. Bạn sẽ thấy một mục với hướng dẫn về tính đồng thời.


In [ ]:
await show_history(session_1)

## Phiên 1: Theo dõi về Mô hình Dữ liệu

Tiếp theo, chúng ta đặt câu hỏi, "Khi nào tôi nên chọn dataclasses thay vì Pydantic?" sử dụng cùng một id phiên. Cognee nên kết hợp các nguyên tắc Python cùng với các cuộc thảo luận FastAPI trước đó để đưa ra lời khuyên chi tiết—cho thấy rằng ngữ cảnh được duy trì trong một phiên có tên.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Xác nhận Lịch sử Phiên 1 Chứa Cả Hai Lượt

Một lần gọi `show_history(session_1)` khác nên hiển thị hai mục Hỏi & Đáp. Điều này phù hợp với bước "phát lại bộ nhớ" của phòng thí nghiệm Mem0 và chứng minh rằng các lượt bổ sung sẽ mở rộng cùng một bản ghi.


In [ ]:
await show_history(session_1)

## Phiên 2: Chuỗi Đánh Giá Thiết Kế — Phiên Mới

Để thể hiện sự tách biệt giữa các chuỗi, chúng tôi khởi chạy `design-review-session` và yêu cầu hướng dẫn ghi nhật ký cho các đánh giá sự cố. Mặc dù cơ sở kiến thức cơ bản là giống nhau, nhưng id phiên mới giữ cho các bản ghi chép được tách biệt.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Xem lại Phiên 2 Lịch sử

`show_history(session_2)` chỉ nên liệt kê cặp yêu cầu/phản hồi trong buổi đánh giá thiết kế. So sánh với Phiên 1 để làm rõ cách Cognee giữ các bản ghi độc lập trong khi tái sử dụng đồ thị tri thức chung.


In [ ]:
await show_history(session_2)

## Tóm tắt

Chúc mừng bạn! Bạn vừa trang bị cho trợ lý lập trình của mình một lớp trí nhớ dài hạn thực sự nhờ vào Cognee.

Trong hướng dẫn này, bạn đã lấy nội dung thô của nhà phát triển (mã nguồn, tài liệu, cuộc trò chuyện) và biến nó thành một đồ thị + trí nhớ vector mà tác nhân của bạn có thể tìm kiếm, suy luận và cải thiện liên tục.

Những gì bạn đã học được

1. **Từ văn bản thô đến trí nhớ AI**: Cách Cognee tiếp nhận dữ liệu không có cấu trúc và biến nó thành trí nhớ thông minh, có thể tìm kiếm được bằng cách sử dụng kiến trúc kết hợp giữa vector + đồ thị tri thức.

2. **Làm giàu đồ thị với memify**: Cách vượt qua việc tạo đồ thị cơ bản và sử dụng memify để thêm các thông tin suy diễn và mối quan hệ phong phú hơn vào đồ thị hiện có của bạn.

3. **Nhiều chiến lược tìm kiếm**: Cách truy vấn trí nhớ với các loại tìm kiếm khác nhau (Hỏi & Đáp nhận thức đồ thị, hoàn thành kiểu RAG, thông tin chi tiết, đoạn thô, tìm kiếm mã, v.v.) tùy thuộc vào nhu cầu của tác nhân.

4. **Khám phá trực quan**: Cách kiểm tra và gỡ lỗi những gì Cognee đã xây dựng bằng cách sử dụng hình ảnh hóa đồ thị và giao diện người dùng Cognee, để bạn có thể thực sự thấy cách kiến thức được cấu trúc.

5. **Trí nhớ nhận thức theo phiên**: Cách kết hợp ngữ cảnh theo từng phiên với trí nhớ ngữ nghĩa liên tục để các tác nhân có thể ghi nhớ qua các lần chạy mà không làm rò rỉ thông tin giữa các người dùng.


## Những Điểm Chính
1. Bộ nhớ như một Đồ thị Kiến thức được hỗ trợ bởi Embeddings

    - **Hiểu biết có cấu trúc**: Cognee kết hợp một kho vector và một kho đồ thị để dữ liệu của bạn vừa có thể tìm kiếm theo ý nghĩa vừa được kết nối bởi các mối quan hệ. Cognee mặc định sử dụng cơ sở dữ liệu dựa trên tệp (LanceDB cho vector, Kuzu cho cơ sở dữ liệu đồ thị).

    - **Truy xuất nhận thức mối quan hệ**: Câu trả lời không chỉ dựa trên “văn bản tương tự” mà còn dựa vào cách các thực thể liên kết với nhau.

    - **Bộ nhớ sống động**: Lớp bộ nhớ phát triển, mở rộng và luôn có thể truy vấn như một đồ thị được kết nối.

2. Chế độ Tìm kiếm & Lý luận
    - **Truy xuất kết hợp**: Tìm kiếm kết hợp sự tương đồng vector, cấu trúc đồ thị và lý luận LLM, từ việc tra cứu đoạn văn bản thô đến trả lời câu hỏi nhận thức đồ thị.

    - **Phù hợp chế độ với công việc**: Sử dụng chế độ kiểu hoàn thành khi bạn muốn câu trả lời ngôn ngữ tự nhiên, và chế độ đoạn văn/tóm tắt/đồ thị khi tác nhân của bạn cần ngữ cảnh thô hoặc để tự đưa ra lý luận.

3. Tác nhân Cá nhân hóa, Nhận thức Phiên làm việc
    - **Ngữ cảnh phiên làm việc + bộ nhớ dài hạn**: Cognee giữ ngữ cảnh “chuỗi” ngắn hạn tách biệt với bộ nhớ dài hạn ở cấp độ người dùng hoặc tổ chức.

## Ứng Dụng Thực Tế

1. **Tác nhân AI Chuyên ngành**

    Sử dụng mẫu từ notebook này để cung cấp năng lượng cho các trợ lý thông minh theo lĩnh vực, hoạt động dựa trên Cognee như lõi truy xuất và lý luận của chúng:

- **Trợ lý nhà phát triển**: Xem xét mã, phân tích sự cố, và trợ lý kiến trúc có thể duyệt qua mã, API, tài liệu thiết kế, và các ticket như một đồ thị bộ nhớ duy nhất.

- **Trợ lý hướng khách hàng**: Các tác nhân hỗ trợ hoặc thành công có thể truy xuất từ tài liệu sản phẩm, FAQ, ghi chú CRM, và các ticket trước đây với truy xuất nhận thức đồ thị và câu trả lời có trích dẫn.

- **Trợ lý chuyên gia nội bộ**: Trợ lý về chính sách, pháp lý, hoặc bảo mật có thể lý luận dựa trên các quy tắc, hướng dẫn, và quyết định lịch sử được kết nối thay vì các tệp PDF riêng lẻ.

    Cognee được định vị rõ ràng như bộ nhớ chính xác, lâu dài cho các tác nhân AI, cung cấp một đồ thị kiến thức sống động thay thế các kết hợp tùy chỉnh giữa kho vector và mã đồ thị.

2. **Hợp nhất Các Kho Dữ liệu thành Một Bộ nhớ**

    Cách tiếp cận tương tự cũng giúp bạn xây dựng một lớp bộ nhớ thống nhất trên các nguồn phân tán:

- **Từ các kho dữ liệu đến một đồ thị**: Nhập dữ liệu có cấu trúc (ví dụ: cơ sở dữ liệu) và dữ liệu không cấu trúc (ví dụ: tài liệu, cuộc trò chuyện) vào một đồ thị duy nhất được hỗ trợ bởi embeddings, thay vì các chỉ mục riêng biệt cho từng hệ thống.

- **Lý luận qua các nguồn với trích dẫn**: Thực hiện lý luận nhiều bước trên tất cả—“kết hợp” nhật ký, số liệu, và tài liệu qua đồ thị—và vẫn trả về câu trả lời có căn cứ với nguồn gốc.

- **Trung tâm kiến thức**: Đối với các lĩnh vực như ngân hàng hoặc giáo dục, Cognee đã được sử dụng để hợp nhất các tệp PDF, hệ thống nội bộ, và dữ liệu ứng dụng thành một đồ thị kiến thức với vector để các tác nhân có thể trả lời câu hỏi với ngữ cảnh chính xác, có trích dẫn.

## Các Bước Tiếp Theo

Bạn đã triển khai vòng lặp bộ nhớ cốt lõi. Dưới đây là các phần mở rộng tự nhiên bạn có thể thử (xem [tài liệu Cognee](https://docs.cognee.ai/) để biết chi tiết):

1. **Thử nghiệm với nhận thức thời gian**: Bật tính năng nhận thức thời gian để trích xuất sự kiện và dấu thời gian từ văn bản.

2. **Giới thiệu lý luận dựa trên ontology**: Xác định một ontology OWL cho lĩnh vực của bạn. Sử dụng hỗ trợ ontology của Cognee để các thực thể và mối quan hệ được trích xuất được gắn vào schema đó, cải thiện chất lượng đồ thị và câu trả lời cụ thể theo lĩnh vực.

3. **Thêm vòng phản hồi**: Để Cognee điều chỉnh trọng số cạnh đồ thị từ phản hồi thực tế của người dùng, giúp truy xuất cải thiện theo thời gian thay vì giữ nguyên.

4. **Tùy chỉnh cho cá nhân hóa & hành vi phiên làm việc**: Sử dụng ID người dùng, tenant, và tập dữ liệu để cung cấp cho mỗi người hoặc nhóm một góc nhìn riêng trên động cơ bộ nhớ chia sẻ.

5. **Mở rộng ra các tác nhân phức tạp hơn**: Kết nối Cognee vào các framework tác nhân để xây dựng hệ thống đa tác nhân, tất cả đều chia sẻ cùng một lớp bộ nhớ. *Microsoft Agent Framework x Cognee plugin sẽ sớm ra mắt.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
